[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pun2beam/makePeriodicTableEPUB/blob/main/notebooks/colab_build.ipynb)

# Periodic Table EPUB build (Colab)
This notebook reproduces the full EPUB build pipeline described in [`spec.md`](../spec.md) using Google Colab's default Python runtime.
**Usage notes**:
- Open this notebook in Google Colab (File → Open notebook → GitHub).
- (Optional) Connect your Google Drive if you want to persist the generated EPUB/cover files.
- Run the cells from top to bottom; the pipeline will download the latest Wikipedia data, normalize it, fetch per-element summaries, build the SVG cover, rasterize it, and assemble the EPUB package inside `book/dist/`.


In [6]:
#@title Install Python dependencies
%cd /content/
!git clone https://github.com/pun2beam/makePeriodicTableEPUB.git
%cd /content/makePeriodicTableEPUB
!pip install --quiet -r requirements.txt

Cloning into 'makePeriodicTableEPUB'...
remote: Enumerating objects: 365, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 365 (delta 11), reused 0 (delta 0), pack-reused 328 (from 2)
Receiving objects: 100% (365/365), 469.62 KiB | 3.24 MiB/s, done.
Resolving deltas: 100% (189/189), done.
/content/makePeriodicTableEPUB


In [8]:
#@title Install system fonts for Japanese text
!apt-get update -y
!apt-get install -y fonts-noto-cjk
!fc-cache -fv


Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,632 B in 1s (2,929 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [20]:
%cd /content/makePeriodicTableEPUB
!git pull

/content/makePeriodicTableEPUB
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 6.02 KiB | 3.01 MiB/s, done.
From https://github.com/pun2beam/makePeriodicTableEPUB
   0b0e4ea..838a552  main       -> origin/main
 * [new branch]      codex/fix-metadata-language-error-in-epub -> origin/codex/fix-metadata-language-error-in-epub
Updating 0b0e4ea..838a552
Fast-forward
 scripts/build_epub.py | 29 ++++++++++++++++++++++++++++-
 1 file changed, 28 insertions(+), 1 deletion(-)


## Optional: Mount Google Drive
Uncomment the cell below if you want the generated assets to be saved to Google Drive instead of the Colab ephemeral filesystem.

In [14]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/path/to/your/workdir

## Build the EPUB
The commands below follow the workflow defined in Section 4 of the specification. They can be rerun to refresh the data or rebuild the package after modifying templates or scripts.

In [24]:
#@title Run data fetch, normalization, and build scripts
# !python scripts/fetch_wiki.py --lang en --page "List of chemical elements" --output-file list-of-chemical-elements-en-rest.json
!python scripts/fetch_wiki.py --lang ja --page "元素の一覧" --output-file yuan-su-noyi-lan-ja-rest.json
# !python scripts/normalize.py
!python scripts/normalize.py --input data/raw/yuan-su-noyi-lan-ja-rest.json
!python scripts/fetch_wiki.py --lang ja --page "元素の一覧" --elements-from data/tables.json --elements-json data/elements.ja.json
!python scripts/build_cover_svg.py --in data/tables.json --out assets/gen/cover.svg
!python scripts/license_attribution.py --out book/OEBPS/attribution.xhtml
!python scripts/rasterize_cover.py --in assets/gen/cover.svg --out book/dist/cover_2560x1600.jpg --width 1600 --height 2560
# !python scripts/build_epub.py --cover book/dist/cover_2560x1600.jpg --out book/dist/PeriodicTable.en.epub --element-data data/elements.en.json
!python scripts/build_epub.py --cover book/dist/cover_2560x1600.jpg --out book/dist/PeriodicTable.ja.epub --element-data data/elements.ja.json


2025-09-23 19:02:12,322 INFO __main__: Starting fetch
2025-09-23 19:02:12,707 INFO __main__: Fetched REST API page html
2025-09-23 19:02:12,712 INFO __main__: Saved raw data
Wrote normalized data to data/tables.json
2025-09-23 19:02:14,135 INFO __main__: Starting fetch
2025-09-23 19:02:14,356 INFO __main__: Fetched REST API page html
2025-09-23 19:02:14,361 INFO __main__: Saved raw data
2025-09-23 19:02:14,431 INFO __main__: Fetched REST API summary
2025-09-23 19:02:14,432 INFO __main__: Fetched element summary
2025-09-23 19:02:14,499 INFO __main__: Fetched REST API summary
2025-09-23 19:02:14,500 INFO __main__: Fetched element summary
2025-09-23 19:02:14,567 INFO __main__: Fetched REST API summary
2025-09-23 19:02:14,568 INFO __main__: Fetched element summary
2025-09-23 19:02:14,633 INFO __main__: Fetched REST API summary
2025-09-23 19:02:14,634 INFO __main__: Fetched element summary
2025-09-23 19:02:14,702 INFO __main__: Fetched REST API summary
2025-09-23 19:02:14,703 INFO __main__:

## Inspect generated files
Run the following cell to list the distribution folder and confirm that the EPUB and cover JPEG were produced successfully.

In [16]:
#@title Check build outputs
!ls -lh book/dist

ls: cannot access 'book/dist': No such file or directory


In [23]:
!rm -rf /content/makePeriodicTableEPUB/data
!rm -rf /content/makePeriodicTableEPUB/book/
!rm -rf /content/makePeriodicTableEPUB/makePeriodicTableEPUB/
